This script plots the estimated cell abundance of cell types across tissue (for one sample). Each cell type has a individual subplot. 

It is set up so that all immune cell types are shown together, and endothelial and micro-PVM are shown together. 

Regarding the color scale of subplots, there are three options:
1) Each subplot has its own color scale.
2) All subplots (cell types) on one plot have the same color scale, but this scale differs from sample to sample.
3) All subplots (cell types) on one plot have the same color scale, and all samples have the same color scale. In this case, the color scale is manually set.

**Author:** Yiqing Wang

**Date:** 2024-6-25

INPUT: spatially mapped AnnData

OUTPUT: plot of spatial abundance of selected cell types of one sample

1. Import libraries

In [ ]:
import os
import scanpy as sc
import matplotlib as mpl
from cell2location.utils import select_slide
import numpy as np

2. Read spatially mapped AnnData object

In [ ]:
dir = "path/to/data"
os.chdir(dir)
results_folder = "./test_results/"
sample = "D1"
run_name = f"{results_folder}/{sample}_run_name/"

adata_file = f"{run_name}/sp_mapped.h5ad"
adata_vis = sc.read_h5ad(adata_file)

In [ ]:
# Add 5% quantile ("at least this amount is present") of the posterior distribution of estimated cell abundances,
# to adata.obs with nice names for plotting
# Note that 5% quantile values are used here, but mean values can also be used. 
# Both should be tried, and choose the one that produce the most meaningful plots. 
adata_vis.obs[adata_vis.uns["mod"]["factor_names"]] = adata_vis.obsm[
    "q05_cell_abundance_w_sf"
] # adata_vis.uns["mod"]["factor_names"] saves names of cell types

3. Prepare for plotting spatial cell abundance

In [ ]:
# Select one slide
slide = select_slide(adata_vis, sample) # this is not necessary if there is only one sample in the AnnData

In [ ]:
# Display available cell types
cell_types = adata_vis.uns["mod"]["factor_names"]
print(f"Cell types available: {cell_types}")

In [ ]:
# Select cell types to plot

# select_cell_types = [
#     "B cells",
#     "NK/NKT cells",
#     "T cells",
#     "BAM",
#     "cDC",
#     "Monocytes",
#     "Neutrophils",
# ] # immune cell types

select_cell_types = [
    "Endo",
    "Micro-PVM",
] # endothelial and micro-PVM cell types

4. Plot spatial cell abundance

The color scale options are controlled by the vmax parameter of sc.pl.spatial.

Option 1: each subplot (cell types) has its own color scale

In [ ]:
with mpl.rc_context({"axes.facecolor": "black", "figure.figsize": [4.5, 5]}):

    sc.pl.spatial(
        slide,
        cmap="magma",

        # show select cell types
        color=select_cell_types,

        ncols=4, # can be customized
        size=1.3,
        img_key="hires",

        # limit color scale at 99.2% quantile of cell abundance for each indicidual cell type
        vmin=0,
        vmax="p99.2",
    )

Option 2: all subplots (cell types) have the same scale, but different samples have different scales

In [ ]:
# Make data frame from adata_vis.obs with cell abundance columns of select cell types
cell_abundance = adata_vis.obs[select_cell_types]
all_abundance_values = cell_abundance.values.flatten()

# Get the 99.2% quantile of cell abundance values of select cell types
quantile_max = np.quantile(all_abundance_values, 0.992)
print(f"99.2% quantile of cell abundance values of immune cells: {quantile_max}")

In [ ]:
with mpl.rc_context({"axes.facecolor": "black", "figure.figsize": [4.5, 5]}):

    sc.pl.spatial(
        slide,
        cmap="magma",

        # show select cell types
        color=select_cell_types,

        ncols=4,
        size=1.3,
        img_key="hires",

        # limit color scale for all cell types, at 99.2% quantile of cell abundance values of all cell types
        vmin=0,
        vmax=quantile_max,
    )

Option 3: all subplots (cell types) have the same scale, and all samples have the same scale too

Here, the upper limit value is set manually.

In [ ]:
# This value can be set at the max (or 99.2% percentile) of the cell abundance values of all cell types, of all samples.
all_sample_max = 20

In [ ]:
with mpl.rc_context({"axes.facecolor": "black", "figure.figsize": [4.5, 5]}):

    sc.pl.spatial(
        slide,
        cmap="magma",

        # show select cell types
        color=select_cell_types,

        ncols=4,
        size=1.3,
        img_key="hires",

        # limit color scale for all cell types to a predefined value
        vmin=0,
        vmax=all_sample_max,
    )